In [37]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from tqdm.notebook import tqdm

Получаем кол-во страниц и ссылки на объявления с первой страницы

In [38]:
# URL для парсинга и заголовки
url = 'https://kolesa.kz/cars/vaz/2190/'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36'}

In [39]:
#  Отправляем GET-запрос
response = requests.get(url, headers=headers)

#  Проверяем, успешно ли загрузилась страница
print(f'Статус-код запроса: {response.status_code}')

Статус-код запроса: 200


In [40]:
#  Записываем html страницы в файл
with open("response.txt", "w", encoding="utf-8") as file:
    file.write(response.text)

#  Записываем html в переменную (первая страница)
cod1 = response.text

In [41]:
#  Вытаскиваем ссылки на объявления с первой страницы
parser = BeautifulSoup(cod1, 'html.parser')
link = []

for teg in parser.select('a[class="a-card__link"]'): #  Вытаскиваем теги с ссылками и перебираем их
        link.append('https://kolesa.kz' + teg['href']) #  Вытаскиваем ссылку из тега и добавляем в список
        link = list(set(link)) #  Удаляем повторения (ссылки повторяются по 2 раза)

print(f'Количество объявлений на странице: {len(link)}')

Количество объявлений на странице: 20


In [42]:
#  Находим теги с ссылками на страницы, чтобы узнать общее кол-во страниц
pages = parser.find_all("a", href=lambda href: href and "/cars/vaz/2190/?page=" in href)

for page in pages[:-1]: # перебираем теги кроме последнего (предпоследнее значение это последняя страница)
    quantity_pages = page.text # вытаскиваем номер страницы и переприсваиваем переменной

quantity_pages = int(quantity_pages) # переводим в число (кол-во страниц)
print(f'Количество страниц: {quantity_pages}')

Количество страниц: 115


Перебираем все страницы и получаем ссылки на все объявления

In [44]:
page_error = {}

#  Базовая ссылка на страницу
url_base = 'https://kolesa.kz/cars/vaz/2190/?page='

#  Перебираем все страницы со второй
for page in tqdm(range(2, quantity_pages + 1), desc='Перебор страниц', unit='pages'):
   url = f'{url_base}{page}' #  Формируем ссылку для конкретной страницы
   response = requests.get(url, headers=headers)
   if response.status_code == 200: #  Если все ок - сохраняем код страницы
      cod_page = response.text

      parser = BeautifulSoup(cod_page, 'html.parser')
      new_link = []

      #  Вытаскиваем все ссылки на объявления со страниц и собираем в список
      for teg in parser.select('a[class="a-card__link"]'):
        new_link.append('https://kolesa.kz' + teg['href'])
        new_link = list(set(new_link))
      link = link + new_link

      #  Рандомная задержка от 1 до 3 секунд
      delay = random.uniform(1, 3)
      print(f"Ждем {delay:.2f} секунд...")
      time.sleep(delay)

   else:
      print(f'Код ошибки: {response.status_code}') #  Выводим код ошибки если что-то пошло не так
      page_error.update({'page': page, 'error': response.status_code})

print(f'Кол-во объявлений: {len(link)}')
print(f'Страниц с ошибками: {len(page_error)}')

Перебор страниц:   0%|          | 0/114 [00:00<?, ?pages/s]

Ждем 2.41 секунд...
Ждем 1.87 секунд...
Ждем 1.20 секунд...
Ждем 1.54 секунд...
Ждем 1.41 секунд...
Ждем 2.22 секунд...
Ждем 2.14 секунд...
Ждем 2.91 секунд...
Ждем 2.12 секунд...
Ждем 1.79 секунд...
Ждем 2.38 секунд...
Ждем 1.32 секунд...
Ждем 1.64 секунд...
Ждем 2.79 секунд...
Ждем 1.36 секунд...
Ждем 2.07 секунд...
Ждем 1.79 секунд...
Ждем 2.45 секунд...
Ждем 2.59 секунд...
Ждем 1.47 секунд...
Ждем 1.38 секунд...
Ждем 1.46 секунд...
Ждем 1.98 секунд...
Ждем 1.73 секунд...
Ждем 1.24 секунд...
Ждем 1.81 секунд...
Ждем 2.62 секунд...
Ждем 2.19 секунд...
Ждем 1.97 секунд...
Ждем 2.51 секунд...
Ждем 1.32 секунд...
Ждем 1.53 секунд...
Ждем 1.87 секунд...
Ждем 1.69 секунд...
Ждем 1.49 секунд...
Ждем 2.15 секунд...
Ждем 2.18 секунд...
Ждем 2.99 секунд...
Ждем 1.64 секунд...
Ждем 1.68 секунд...
Ждем 1.62 секунд...
Ждем 2.98 секунд...
Ждем 1.15 секунд...
Ждем 2.85 секунд...
Ждем 1.12 секунд...
Ждем 1.44 секунд...
Ждем 1.29 секунд...
Ждем 2.66 секунд...
Ждем 1.48 секунд...
Ждем 2.45 секунд...


In [ ]:
#  Сохраняем ссылки
auto_links = pd.Series(link)
auto_links.to_csv('auto_links_new.csv', index=False)

Парсим характеристики авто из объявлений

In [51]:
#  Проверка есть ли на странице каптча, или временная блокировка
def check_captcha(url_i):
    while True:
        response = requests.get(url_i, headers=headers, timeout=15)
        if response.status_code == 429:
            print("Слишком много запросов! Жду 10 минут...")
            time.sleep(600)
            continue
        elif 'captcha' in response.text.lower() and len(response.text) < 5000:
            print("Капча обнаружена! Жду 10 минут...")
            time.sleep(600)  # Ждём 10 минут и пробуем заново
            continue

        else:
            return response

In [60]:
all_params = []

In [ ]:
#  Характеристики
title = ['Поколение', 'Кузов', 'Объем двигателя, л', 'Пробег', 'Коробка передач', 'Привод', 'Руль', 'Цвет', 'Растаможен в Казахстане']

#  Перебираем объявления
for i, url_i in tqdm(enumerate(link), desc='Парсинг объявлений', unit='obj', total=len(link)):
    print(f'Объявление: {i + 1}')
    params_i = []
    response = check_captcha(url_i) #  Проверяем каптчу и блокировку

    if response.status_code == 404:
        print('Страница не найдена!')
        not_found += 1
        continue
    
    elif response.status_code == 200:
        code = response.text
        parser = BeautifulSoup(code, 'html.parser')

        model_teg = parser.select_one('span[itemprop="brand"]') # Тег - модель
        params_i.append(model_teg.text.strip()) 
        year_teg = parser.select_one('span[class="year"]') # Тег - год
        params_i.append(year_teg.text.strip()) 
        city_teg = parser.select_one('dt[class="value-title"]') # Тег - город
        if city_teg: 
            value_teg = city_teg.find_next_sibling('dd')
            params_i.append(value_teg.text.strip())
        else:
            params_i.append(None)
        
        for param in title: #  Перебираем теги с параметрами
            dt_teg = parser.select_one(f'dt[title="{param}"]')
            if dt_teg:
                dd_teg = dt_teg.find_next_sibling('dd')
                params_i.append(dd_teg.text.strip())
            else:
                params_i.append(None) #  Если параметра нет, заполняем None

        options_teg = parser.select('span[class="offer__option-label"]') # Тег - опции
        if options_teg:
            options_i = []
            for option in options_teg: 
                options_i.append(option.text.strip())
            params_i.append(', '.join(options_i))
        else:
            params_i.append(None) #  Если опций нет, заполняем None
        
        #  Метка с состоянием авто
        new_teg = parser.select_one('span[class="a-labels__item a-labels__item--new"]') 
        avaria_teg = parser.select_one('div[class="offer__parameters-mortgaged"]')
        if new_teg:
            params_i.append(new_teg.text.strip())
        elif avaria_teg:
            params_i.append(avaria_teg.text.strip())
        else:
            params_i.append('б/у')
        
        price_teg = parser.select_one('div[data-test="offer-price"]') # Тег - цена
        params_i.append(price_teg.text.replace("\xa0", "").replace("\n", "").replace("₸", "").strip())
        
        #  Сохраняем все в список
        all_params.append(params_i)
        print(f'Собрано {len(all_params)} объявлений')

        #  Рандомная задержка от 5 до 10 секунд
        delay = random.uniform(5, 10)
        print(f"Ждем {delay:.2f} секунд...")
        time.sleep(delay)

        #  Рандомная задержка от 5 до 10 минут, каждые 500 итераций
        if (i + 1) % 500 == 0:
            delay = random.uniform(600, 1200)
            print(f"Пауза {delay // 60} минут...")
            time.sleep(delay)

    else:
        print(f'Остановка парсера, ошибка: {response.status_code}')
        print(f'Последний URL: {url_i}')
        break

Парсинг объявлений:   0%|          | 0/2297 [00:00<?, ?obj/s]

Объявление: 1
Собрано 1 объявлений
Ждем 6.13 секунд...
Объявление: 2
Собрано 2 объявлений
Ждем 5.29 секунд...
Объявление: 3
Собрано 3 объявлений
Ждем 5.28 секунд...
Объявление: 4
Собрано 4 объявлений
Ждем 7.48 секунд...
Объявление: 5
Собрано 5 объявлений
Ждем 7.43 секунд...
Объявление: 6
Собрано 6 объявлений
Ждем 5.05 секунд...
Объявление: 7
Собрано 7 объявлений
Ждем 8.62 секунд...
Объявление: 8
Собрано 8 объявлений
Ждем 8.07 секунд...
Объявление: 9
Собрано 9 объявлений
Ждем 7.90 секунд...
Объявление: 10
Собрано 10 объявлений
Ждем 6.28 секунд...
Объявление: 11
Собрано 11 объявлений
Ждем 9.87 секунд...
Объявление: 12
Собрано 12 объявлений
Ждем 8.98 секунд...
Объявление: 13
Собрано 13 объявлений
Ждем 5.40 секунд...
Объявление: 14
Собрано 14 объявлений
Ждем 8.87 секунд...
Объявление: 15
Собрано 15 объявлений
Ждем 7.72 секунд...
Объявление: 16
Собрано 16 объявлений
Ждем 7.17 секунд...


KeyboardInterrupt: 

Сохраняем объявления с характеристиками

In [71]:
columns = ['Модель','Год','Город','Поколение','Кузов','Объем двигателя, л','Пробег','Коробка передач',
           'Привод','Руль','Цвет','Растаможен в Казахстане','Опции','Состояние','Цена']

all_params_df = pd.DataFrame(all_params, columns=columns)
all_params_df.head()

all_params_df.to_csv('all_params.csv', sep=',', index=False)